# ReAct Agent - Código Python com Anotações de Estudo


Este notebook foi desenvolvido baseado no tutorial indicado Fastcamp de Agentes Inteligentes: https://www.youtube.com/watch?v=hKVhRA9kfeM


In [10]:
pip install groq python-dotenv # Instale as bibliotecas necessárias

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [11]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

In [12]:
# 1. CONFIGURAÇÃO DA API GROQ
# Carregando variáveis de ambiente do arquivo .env
# Isso é uma boa prática para não expor chaves de API no código
load_dotenv()

# Criando o cliente da API Groq
# IMPORTANTE: Preciso sempre ter o arquivo .env na raiz do projeto
# com GROQ_API_KEY=minha_chave_aqui
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),  # Busca a chave nas variáveis de ambiente
)

In [13]:
class Agent:
    """
    Classe principal que implementa um agente conversacional.
    
    - O agente mantém um histórico de conversa (self.messages)
    - Usa o padrão __call__ para ser chamado como uma função
    - Sempre adiciona system prompt no início
    """
    
    def __init__(self, client, system):
        # client é a conexão com a API, system é o "cérebro" do agente
        self.client = client  # Guarda a conexão com a API
        self.system = system  # System prompt que define como o agente se comporta
        self.messages = []    # Lista que armazena todo o histórico da conversa
        
        # Se tiver system prompt, adiciona como primeira mensagem
        # DICA: O system prompt é SEMPRE a primeira mensagem da conversa
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

In [ ]:
def __call__(self, message=""):
    """
    Método mágico que permite chamar o agente como: agent("pergunta")
    APRENDI: __call__ é o que permite fazer agent() ao invés de agent.execute()
    """
    # Se enviou uma mensagem, adiciona no histórico como "user"
    if message:
        self.messages.append({"role": "user", "content": message})
    
    # Executa a chamada para a API e pega a resposta
    result = self.execute()
    
    # Adiciona a resposta do agente no histórico como "assistant"
    # IMPORTANTE: Sempre manter o histórico atualizado!
    self.messages.append({"role": "assistant", "content": result})
    return result

In [ ]:
def execute(self):
    """
    Faz a chamada real para a API do Groq
    APRENDIZADO: Aqui é onde a "mágica" acontece - enviamos o histórico
    e recebemos a próxima resposta do modelo
    """
    completion = self.client.chat.completions.create(
        messages=self.messages,           # Envia TODO o histórico
        model="llama-3.3-70b-versatile"  # Modelo que estamos usando
    )
    # Retorna apenas o texto da resposta (sem metadados)
    return completion.choices[0].message.content

**Observação:** Para expandir minha prática, decidi trabalhar em um domínio diferente do abordado na videoaula. No meu código, usarei tools e um system prompt focado em tirar dúvidas sobre as tarefas do Fastcamp de Agentes Inteligentes.

In [21]:
# CONCEITO: O system prompt é como dar um "manual de instruções"
# para o agente. Ele define:
# - Como o agente deve pensar (Thought)
# - Quais ações pode executar (Action) 
# - Como deve pausar e aguardar resultados (PAUSE)
# - Como processar observações (Observation)
# - Como dar a resposta final (Answer)

system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_activity_title:
e.g. get_activity_title: 1
Returns the title of the activity from Fastcamp de Agentes Inteligentes based on the number

get_activity_deadline:
e.g. get_activity_deadline: Vídeo: Fundamentos de Agentes de IA (I)
Returns the deadline for the activity based on its title

Example session:

Question: What is the deadline for activity 1?
Thought: I need to first get the title of activity 1, then find its deadline
Action: get_activity_title: 1
PAUSE 

You will be called again with this:

Observation: Vídeo: Fundamentos de Agentes de IA (I)

Thought: Now I need to get the deadline for this activity
Action: get_activity_deadline: Vídeo: Fundamentos de Agentes de IA (I)
PAUSE

You will be called again with this: 

Observation: 2 dias

If you have the answer, output it as the Answer.

Answer: The deadline for activity 1 is 2 days.

Now it's your turn:
""".strip()

In [22]:
# CONCEITO: Tools são funções Python que o agente pode "chamar"
# Elas expandem as capacidades do LLM beyond just generating text

def get_activity_title(activity_number: str) -> str:
    """
    TOOL 1: Busca o título da atividade pelo número
    
    APRENDIZADO: Esta função simula uma "base de dados" das atividades
    Em um sistema real, isso poderia consultar um banco de dados
    """
    # Dicionário que mapeia números para títulos das atividades
    # DICA: Em produção, isso viria de uma API ou banco de dados
    activities = {
        "1": "Vídeo: Fundamentos de Agentes de IA (I)",
        "2": "Prática: Python: Criando um ReAct Agent do Zero (I)",
        "3": "Prática: Validação de dados com Pydantic (I)",
        "4": "Vídeo: Introdução ao n8n (II)",
        "5": "Prática: Construindo um fluxo n8n(II)",
        "6": "Prática: Embedding (II)",
        "7": "Leitura: n8n (II)",
        "8": "Pratica: Agentes com Google ADK (III)",
        "9": "Leitura: Construindo Agentes Google ADK(III)",
        "10": "Pratica: Multi Agentes com ADK (III)",
        "11": "Prática: Criando agentes com ADK e Streamlit (IV)",
        "12": "Prática: Criando agentes com n8n, ADK e Whatsapp (IV)",
        "13": "Desafio: Orquestrando Agentes com Google ADK (IV)",
        "14": "Prática: Projeto Final (V)"
    }
    # get() é mais seguro que [], retorna valor padrão se não encontrar
    return activities.get(activity_number, "Atividade não encontrada")

def get_activity_deadline(activity_title: str) -> str:
    """
    TOOL 2: Busca o prazo da atividade pelo título
    
    CONCEITO: Esta tool funciona em conjunto com a primeira
    Isso demonstra como tools podem se complementar!
    """
    # Mapeamento de títulos para prazos
    # OBSERVAÇÃO: Note que os títulos devem ser EXATOS (case sensitive)
    deadlines = {
        "Vídeo: Fundamentos de Agentes de IA (I)": "2 dias",
        "Prática: Python: Criando um ReAct Agent do Zero (I)": "4 dias",
        "Prática: Validação de dados com Pydantic (I)": "4 dias",
        "Vídeo: Introdução ao n8n (II)": "3 dias",
        "Prática: Construindo um fluxo n8n(II)": "3 dias",
        "Prática: Embedding (II)": "3 dias",
        "Leitura: n8n (II)": "3 dias",
        "Pratica: Agentes com Google ADK (III)": "4 dias",
        "Leitura: Construindo Agentes Google ADK(III)": "2 dias",
        "Pratica: Multi Agentes com ADK (III)": "4 dias",
        "Prática: Criando agentes com ADK e Streamlit (IV)": "4 dias",
        "Prática: Criando agentes com n8n, ADK e Whatsapp (IV)": "3 dias",
        "Desafio: Orquestrando Agentes com Google ADK (IV)": "5 dias",
        "Prática: Projeto Final (V)": "7 dias"
    }
    return deadlines.get(activity_title, "Prazo não encontrado")


In [24]:
def loop(max_iterations=10, query: str = ""):
    """
    Esta função implementa o CORE do ReAct: o loop automático
    
    FLUXO:
    1. Agente recebe pergunta
    2. Agente pensa (Thought)
    3. Agente decide uma ação (Action) e pausa (PAUSE)
    4. Nós executamos a ação e retornamos observação (Observation)
    5. Repete até ter resposta final (Answer)
    
    ISSO É REVOLUCIONÁRIO! O LLM não apenas gera texto, mas "age" no mundo!
    """
    
    # Cria uma nova instância do agente para cada consulta
    # IMPORTANTE: Cada consulta tem seu próprio histórico limpo
    agent = Agent(client=client, system=system_prompt)
    
    # Lista das tools disponíveis - precisa bater com os nomes das funções!
    tools = ["get_activity_title", "get_activity_deadline"]
    
    next_prompt = query  # Primeira mensagem é a pergunta do usuário
    i = 0  # Contador de iterações para evitar loops infinitos
  
    while i < max_iterations:
        i += 1
        
        # Envia a mensagem para o agente e recebe resposta
        result = agent(next_prompt)
        print(result)  # Mostra o que o agente está "pensando"

        # PADRÃO REACT: Verifica se agente quer executar uma ação
        if "PAUSE" in result and "Action" in result:
            # USA REGEX para extrair a ação! Isso é muito inteligente
            # Busca padrão: "Action: nome_da_funcao: argumento"
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            
            if action:  # Se encontrou uma ação válida
                chosen_tool = action[0][0]  # Nome da função
                arg = action[0][1]          # Argumento para a função

                # Executa a tool SE ela existir na nossa lista
                if chosen_tool in tools:
                    # EVAL é perigoso em produção, mas funciona para estudo!
                    # Em produção, usaria um dispatcher mais seguro
                    result_tool = eval(f"{chosen_tool}('{arg}')")
                    next_prompt = f"Observation: {result_tool}"
                else:
                    next_prompt = "Observation: Tool not found"

                print(next_prompt)  # Mostra o resultado da tool
                continue  # Volta para o início do loop

        # CONDIÇÃO DE PARADA: Agente deu resposta final
        if "Answer" in result:
            break  # Sai do loop, missão cumprida!

## Exemplos de Uso

In [25]:
print("TESTANDO O REACT AGENT")
print("Criado durante os estudos do Fastcamp de Agentes Inteligentes")
print("=" * 60)

# TESTE 1: Pergunta simples
print("TESTE 1: Consultando prazo de uma atividade")
print("-" * 40)
loop(query="What is the deadline for activity 5?")

print("\n" + "=" * 60 + "\n")

# TESTE 2: Pergunta complexa que precisa de múltiplas tools
print("TESTE 2: Consultando múltiplas atividades")
print("-" * 40)
loop(query="What are the deadlines for activity 1 and activity 10? What is the total time for both?")

print("\n" + "=" * 60)
print("Testes concluídos! O agente ReAct está funcionando!")

TESTANDO O REACT AGENT
Criado durante os estudos do Fastcamp de Agentes Inteligentes
TESTE 1: Consultando prazo de uma atividade
----------------------------------------


TypeError: 'Agent' object is not callable

PRINCIPAIS CONCEITOS QUE APRENDI:

1. REACT = Reasoning (Thought) + Acting (Action)
   - LLM não só gera texto, mas pode "agir" no mundo
   - Cada ação é seguida de uma observação
   - Permite resolver problemas complexos passo a passo

2. SYSTEM PROMPT é fundamental
   - Define o "comportamento" do agente
   - Exemplos são cruciais para ensinar o padrão
   - Deve ser claro e detalhado

3. TOOLS expandem capacidades
   - Funções Python que o agente pode "chamar"
   - Podem acessar APIs, bancos de dados, etc.
   - Tornam o LLM mais poderoso que apenas geração de texto

4. LOOP é onde a mágica acontece
   - Automatiza a execução das actions
   - Simula um humano "assistindo" o agente trabalhar
   - Permite iteração até encontrar a resposta

5. HISTÓRICO DE CONVERSA
   - Mantém contexto durante toda a interação
   - Permite que o agente "lembre" de conversas anteriores
   - Essencial para conversas coerentes